In [9]:
import os
from langchain_community.embeddings import AzureOpenAIEmbeddings

# 環境変数からAPIキーを取得
api_key = os.getenv("AZURE_OPENAI_KEY")

# Azureのエンドポイントとデプロイメント名
azure_openai_base = "https://aoai-furutachi-us.openai.azure.com/openai/deployments/"  # エンドポイントに/deploymentsを追加
embedding_model_deployment_name = "text-embedding-3-small"  # デプロイメント名を確認してください

# Embedding APIのバージョンとAPIのタイプを設定
openai_api_version = "2023-05-15"
openai_api_type = "azure"

# AzureOpenAIEmbeddingsを使用して初期化
embeddings = AzureOpenAIEmbeddings(
    openai_api_key=api_key,
    deployment=embedding_model_deployment_name,
    openai_api_base=azure_openai_base,
    openai_api_version=openai_api_version,
    openai_api_type=openai_api_type,
    chunk_size=2048,
    validate_base_url=True  # 追加
)


# テスト用のテキスト
test_text = "This is a test text for embedding."

# エンベディングを取得
try:
    embedding_result = embeddings.embed_query(test_text)
    print("Embedding successfully retrieved!")
    print(embedding_result)  # 結果を出力
except Exception as e:
    print(f"Failed to retrieve embedding: {e}")


Embedding successfully retrieved!
[0.0014321928149825974, 0.016108495042609368, 0.0167747818716437, -0.03305311663258638, -0.011712299303392884, -0.03874922733954452, 0.03190344356127849, -0.013547857956957269, 0.01217608865508952, 0.004713008411586216, 0.041466636755844125, 0.027435391625290677, -0.017441070563323273, -0.024717982208991077, 0.022457827984883896, 0.025946042842606647, -0.008001989146432896, 0.005813690187774425, -0.01541607809481652, 0.051944344060085086, 0.003520875410694998, -0.03584891361119366, -0.014018178674190258, 0.04191082921696384, 0.022875891258567718, -0.04867822543292219, 0.010529964747790128, 0.04279921041391279, 0.04805112865975122, -0.03362795130559508, 0.01758477923157545, -0.03801761381662997, 0.00385401952370413, -0.03287021232053512, -0.01737574945737878, 0.022039764711200074, 0.013547857956957269, 0.04261630982715201, -0.00345881905619606, -0.006793525869055555, -0.01613462423004526, -0.026756040202538398, -0.005104942497834144, 0.01770236057503697

ModuleNotFoundError: No module named 'langchain_azure_openai'

In [1]:
import os
import fitz  # PyMuPDFとしてインポート
import difflib
import re
import unicodedata
from tkinter import Tk, filedialog

# テキスト前処理関数（改行を維持）
def preprocess_text(text):
    # Unicode正規化
    text = unicodedata.normalize('NFKC', text)
    # 不要なスペースを削除（ただし改行は維持）
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()

# 方法A：extract_text_with_blocks
def extract_text_with_blocks(pdf_document):
    page_texts = []
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        # ブロック情報を取得
        blocks = page.get_text("dict")["blocks"]
        text_content = ""
        for block in blocks:
            if block["type"] == 0:  # テキストブロックの場合
                # ライン情報を取得してテキストを結合
                for line in block["lines"]:
                    line_text = ""
                    for span in line["spans"]:
                        line_text += span["text"]
                    text_content += line_text + "\n"  # ラインの終わりで改行
                text_content += "\n"  # ブロックの終わりで改行
        # テキストの前処理を適用
        text_content = preprocess_text(text_content)
        page_texts.append((page_num + 1, text_content))
    return page_texts

# 方法B：extract_text_with_pymupdf4llm
def extract_text_with_pymupdf4llm(pdf_document):
    from pymupdf4llm import to_markdown  # 必要に応じてインポート
    page_texts = []
    for page_num in range(len(pdf_document)):
        # ドキュメント全体を渡し、pagesパラメータでページを指定
        markdown_text = to_markdown(pdf_document, pages=[page_num])
        # テキストの前処理を適用
        markdown_text = preprocess_text(markdown_text)
        page_texts.append((page_num + 1, markdown_text))
    return page_texts

# テキストをファイルに保存する関数
def save_page_texts(page_texts, folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    for page_num, text in page_texts:
        with open(f"{folder_name}/page_{page_num}.txt", "w", encoding="utf-8") as f:
            f.write(text)

# テキストを比較して差分を表示する関数
def compare_texts(texts_a, texts_b):
    for (page_num_a, text_a), (page_num_b, text_b) in zip(texts_a, texts_b):
        if page_num_a != page_num_b:
            print(f"ページ番号が一致しません: 方法Aのページ{page_num_a}, 方法Bのページ{page_num_b}")
            continue
        print(f"\n===== ページ {page_num_a} の比較 =====")
        diff = difflib.unified_diff(
            text_a.splitlines(),
            text_b.splitlines(),
            fromfile='方法A',
            tofile='方法B',
            lineterm=''
        )
        diff_output = '\n'.join(diff)
        if diff_output:
            print(diff_output)
        else:
            print("差分はありません。")

# PDFファイルを選択する関数
def select_pdf_file():
    root = Tk()
    root.withdraw()
    pdf_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
    return pdf_path

# メイン処理
def main():
    # PDFファイルを選択
    pdf_path = select_pdf_file()
    if not pdf_path:
        print("PDFファイルが選択されませんでした。")
        return

    # PDFを開く
    doc = fitz.open(pdf_path)

    # 方法Aでテキストを抽出
    texts_a = extract_text_with_blocks(doc)
    save_page_texts(texts_a, "method_a_texts")

    # 方法Bでテキストを抽出
    texts_b = extract_text_with_pymupdf4llm(doc)
    save_page_texts(texts_b, "method_b_texts")

    # テキストを比較
    compare_texts(texts_a, texts_b)

if __name__ == "__main__":
    main()


Processing C:/Users/km00183/OneDrive - Komatsu Ltd/作業フォルダ/資料改革/work/02_サービス資料UX改革/03_生成AIワーキングチーム/01_品質チェック/サンプル/ShopManual_Sample50Error_JP.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing C:/Users/km00183/OneDrive - Komatsu Ltd/作業フォルダ/資料改革/work/02_サービス資料UX改革/03_生成AIワーキングチーム/01_品質チェック/サンプル/ShopManual_Sample50Error_JP.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing C:/Users/km00183/OneDrive - Komatsu Ltd/作業フォルダ/資料改革/work/02_サービス資料UX改革/03_生成AIワーキングチーム/01_品質チェック/サンプル/ShopManual_Sample50Error_JP.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing C:/Users/km00183/OneDrive - Komatsu Ltd/作業フォルダ/資料改革/work/02_サービス資料UX改革/03_生成AIワーキングチーム/01_品質チェック/サンプル/ShopManual_Sample50Error_JP.pdf...
[  

In [1]:
import fitz  # PyMuPDF
import pymupdf4llm  # PyMuPDF4LLMのインポート

# PDFドキュメントの読み込み
pdf_path = "path/to/your/pdf.pdf"  # ファイルパスを指定
doc = fitz.open(pdf_path)

# マークダウン形式でテキストを抽出
markdown_text = pymupdf4llm.to_markdown(doc)

# マークダウンの内容を表示
print(markdown_text)


Selected file: C:/Users/km00183/OneDrive - Komatsu Ltd/作業フォルダ/資料改革/work/02_サービス資料UX改革/03_生成AIワーキングチーム/01_品質チェック/サンプル/(誤字脱字あり)抜粋版WA470-10_SM_日本語_Sec40.pdf


In [2]:
import fitz  # PyMuPDF
import pymupdf4llm  # PyMuPDF4LLMのインポート
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Tkinterのルートウィンドウを作成し、表示しないようにする
root = Tk()
root.withdraw()

# ファイル選択ダイアログを表示し、選択されたファイルのパスを取得
pdf_path = askopenfilename(filetypes=[("PDF files", "*.pdf")])

# PDFドキュメントの読み込み
if pdf_path:
    doc = fitz.open(pdf_path)

    # マークダウン形式でテキストを抽出
    markdown_text = pymupdf4llm.to_markdown(doc)

    # マークダウンの内容を表示
    print(markdown_text)
else:
    print("No file selected.")

Processing C:/Users/km00183/OneDrive - Komatsu Ltd/作業フォルダ/資料改革/work/02_サービス資料UX改革/03_生成AIワーキングチーム/01_品質チェック/サンプル/(誤字脱字あり)抜粋版WA470-10_SM_日本語_Sec40.pdf...
[                                        ] (0/7====[=====                                   ] (1/=====[===========                             ] (2/=====[=================                       ] (3/====[======================                  ] (4/7=====[============================            ] (5/7=====[==================================      ] (6/7=====[========================================] (7/7]
# H-1 機械が発進しない、または走行速度が遅い

故障現象 機械が発進しない、または走行速度が遅い

            - トランスミション油量、作動油タンク油量が適量であることを確認する

            - 油圧ホース、ポンプモタなどからの油漏れがないか確認する

            - 故障コードが出ている場合は、該当する故障コードの診断を先に実施する

            - 下記センサ類の取り付けにガタや緩みがないか確認する。また、確認後に較正が必要なセンサ類
については較正を実施する

関連情報

T/M 油温センサ、HST 油温センサ、T/M 入力軸回転数センサ、T/M 出力軸回転センサ、ポン
プモータ2回転数センサ、シンクロストロークセンサ、ブレーキペダルストロークセンサ

            - ドライブシャフトなどに異常がないことを確認する

            - パーキングブレーキが確実に解除されていることを

In [2]:
import tiktoken

text = """あなたは建設機械のマニュアルをチェックする専門的なアシスタントです。以下の指示に従って、テキストのチェックを行ってください。
これは技術文書なので、文法については厳密なチェックは不要です。

**チェック項目**:
1. **誤字の確認**: 誤字を指摘してください。
2. **文脈上の誤りの特定**: 記載内容が明確に間違っていて、著しい誤解を招く可能性がある場合を指摘してください。
3. **重大な文法エラーの指摘**: 誤解を生む可能性が高い、重大な文法ミスを指摘してください。

**指摘を行う際のガイドライン**:
- 判断に迷った場合や確信が持てない場合は、偽陽性を避けるため指摘を控えてください。
- **指摘してはいけない項目**:
  - 不自然な改行や空白（PDF抽出時の仕様）
  - 文法的に文章が省略されているが、意味が伝わる場合
  - カタカナ語の長音符に関するもの（独自ルールあり。例：バッテリ、モータ）

**回答形式**:
- 指摘内容は**日本語のJSON形式**で提供してください。
- **JSON以外の文字列は一切含めないでください**。説明や追加情報も不要です。
- 回答では、コードブロックやバッククォート（）は使用しないでください。

**JSONのフォーマット**:

[
  {{
    "page": <ページ番号（整数）>,
    "category": "<カテゴリ>",
    "reason": "<指摘理由>",
    "error_location": "<指摘箇所>",
    "context": "<周辺テキスト>",
    "importance": <重要度（1〜5）>,
    "confidence": <自信度（1〜5）>
  }},
  ...
]

- 各フィールドの説明:
  - "page": 該当ページ番号（整数）
  - "category": 指摘のカテゴリ（例: "誤字", "文法", "文脈"）
  - "reason": 指摘理由（簡潔に、どう修正すべきか）
  - "error_location": 指摘箇所
  - "context": 周辺テキスト
  - "importance": 指摘の重要度（1〜5、5が最も重要）
  - "confidence": 指摘根拠への自信度（1〜5、5が最も自信がある）"""
encoding = tiktoken.encoding_for_model("gpt-4")  # または "gpt-3.5-turbo"
tokens = encoding.encode(text)
print(len(tokens))


787
